# 04 - HTTP/SSE Transports (local test server)

This notebook starts a tiny local JSON-RPC server (HTTP + SSE) and validates the client transports.
It is meant for transport debugging, not as a production MCP server example.


In [ ]:
import sys
from pathlib import Path

def find_root(start: Path) -> Path:
    for cur in [start, *start.parents]:
        if (cur / "pyproject.toml").exists():
            return cur
    return start

ROOT = find_root(Path.cwd().resolve())
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from mcp_tuning import connect_http, connect_sse


In [ ]:
import json
import threading
from http.server import BaseHTTPRequestHandler, ThreadingHTTPServer
from queue import Queue
from typing import Any

def handle(req: dict[str, Any]) -> dict[str, Any] | None:
    method = req.get("method")
    params = req.get("params") or {}
    rid = req.get("id")
    if method == "initialize":
        return {"jsonrpc":"2.0","id":rid,"result": {"protocolVersion":"2024-11-05","serverInfo":{"name":"nb-http","version":"0"},"capabilities":{"tools":{},"resources":{},"prompts":{}}}}
    if method == "tools/list":
        return {"jsonrpc":"2.0","id":rid,"result": {"tools":[{"name":"echo"}]}}
    if method == "tools/call":
        return {"jsonrpc":"2.0","id":rid,"result": {"echo": params.get("arguments") or {}}}
    if rid is None:
        return None
    return {"jsonrpc":"2.0","id":rid,"error": {"code": -32601, "message": "not found"}}

class Server:
    def __init__(self):
        self.queue: "Queue[dict[str, Any]]" = Queue()
        self.stop = threading.Event()
        srv = self
        class H(BaseHTTPRequestHandler):
            protocol_version = "HTTP/1.1"
            def log_message(self, format, *args):
                return
            def do_POST(self):
                if self.path != "/rpc":
                    self.send_error(404); return
                length = int(self.headers.get("Content-Length", "0"))
                raw = self.rfile.read(length)
                req = json.loads(raw.decode("utf-8"))
                resp = handle(req)
                if self.headers.get("X-Async") == "1" and resp is not None:
                    srv.queue.put(resp)
                    self.send_response(202); self.send_header("Content-Length","0"); self.end_headers(); return
                if resp is None:
                    self.send_response(204); self.send_header("Content-Length","0"); self.end_headers(); return
                data = json.dumps(resp, ensure_ascii=False).encode("utf-8")
                self.send_response(200)
                self.send_header("Content-Type","application/json")
                self.send_header("Content-Length", str(len(data)))
                self.end_headers()
                self.wfile.write(data)
            def do_GET(self):
                if self.path != "/sse":
                    self.send_error(404); return
                self.send_response(200)
                self.send_header("Content-Type","text/event-stream")
                self.send_header("Cache-Control","no-cache")
                self.send_header("Connection","keep-alive")
                self.end_headers()
                while not srv.stop.is_set():
                    try:
                        msg = srv.queue.get(timeout=0.2)
                    except Exception:
                        continue
                    payload = f"data: {json.dumps(msg, ensure_ascii=False)}\n\n".encode("utf-8")
                    try:
                        self.wfile.write(payload); self.wfile.flush()
                    except Exception:
                        return
        self.httpd = ThreadingHTTPServer(("127.0.0.1", 0), H)
    @property
    def base(self) -> str:
        host, port = self.httpd.server_address
        return f"http://{host}:{port}"
    def start(self):
        t = threading.Thread(target=self.httpd.serve_forever, daemon=True)
        t.start()
        return t
    def shutdown(self):
        self.stop.set()
        self.httpd.shutdown()

srv = Server()
srv.start()
base = srv.base
base


In [ ]:
# HTTP transport
c = connect_http(f"{base}/rpc")
c.server_info()


In [ ]:
c.tools()
r = c.inspector.call_tool("echo", {"x": 1})
r.ok, r.result


In [ ]:
# SSE transport (server replies via SSE by using X-Async header)
c2 = connect_sse(sse_url=f"{base}/sse", rpc_url=f"{base}/rpc", headers={"X-Async": "1"})
c2.server_info()
r2 = c2.inspector.call_tool("echo", {"x": 2})
r2.ok, r2.result


In [ ]:
# Cleanup
c.close(); c2.close(); srv.shutdown()
